# Summary 
This is a model used to predict the weather in Dunedin, New Zealand. Daily weather dataset from 2016-2019 were used. There are two models made:
+ Full year air temperature prediction
+ Half year air temperature prediction

# Importing moduels and loading datasets

In [12]:
import pandas as pd 
import pygal
from pygal.style import DarkStyle
from IPython.display import SVG, HTML, display
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


six = pd.read_csv('./Weather_Data/daily/2016.csv').rename(columns={'Date(NZST)': 'Date','Tair(C)':'AirTemp'})
print(f'Loaded 2016, Shape:{six.shape}')
sev = pd.read_csv('./Weather_Data/daily/2017.csv').rename(columns={'Date(NZST)': 'Date','Tair(C)':'AirTemp'})
print(f'Loaded 2017, Shape:{sev.shape}')

Xlist = ['AirTemp','RH(%)'] #
Yvar = 'AirTemp'
# RH - relative humidity
# Tair - Temperature of air    <== What we want to predict 
display(six.head(3))
display(sev.head(3))

Loaded 2016, Shape:(717, 6)
Loaded 2017, Shape:(728, 6)


,Station,Date,AirTemp,Twet(C),RH(%),Tdew(C)
0,15752,20160101:0800,16.4,13.0,66.0,10.0
1,3925,20160101:0800,17.1,15.3,82.0,14.0
2,15752,20160102:0800,16.2,13.7,74.0,11.6


,Station,Date,AirTemp,Twet(C),RH(%),Tdew(C)
0,15752,20170101:0800,15.6,14.3,86.0,13.3
1,3925,20170101:0800,18.2,17.2,90.0,16.5
2,15752,20170102:0800,10.4,8.2,72.0,5.6


# Data & Feature engineering
+ cleaning<br>
  - Seprated the data into the two corresponding stations in Dunedin
  - Made sure the data used have the same length as some dates are missing. 
+ visualisation
  - Humidity and Air Temperature have a inverse relationship

In [13]:
#Seperating data by stations
Stations = {
    'One': 15752,
    'Two': 3925
}
sixOne = six[six['Station'] == Stations['One']].reset_index()
sixTwo = six[six['Station'] == Stations['Two']].reset_index()

sevOne = sev[sev['Station'] == Stations['One']].reset_index()
sevTwo = sev[sev['Station'] == Stations['Two']].reset_index()

display(sixOne.shape)
display(sevOne.shape)

(352, 7)

(364, 7)

In [14]:
#Ensure data have same dates and missing days does not affect the model
def clean(X, Y):
    Xdates = [i[4:(len(i)-1)] for i in list(X['Date'])]
    Ydates = [i[4:(len(i)-1)] for i in list(Y['Date'])]
    yfilt = []
    xfilt = []
    for i in Xdates:
        if i in Ydates:
            yfilt.append(Ydates.index(i))
            xfilt.append(Xdates.index(i))

    Yres = Y.iloc[yfilt].reset_index()
    Xres = X.iloc[xfilt].reset_index()

    if Yres.shape == Xres.shape:
        return (Yres,Xres)
    else:
        print('Filter Error.')

(Yone, Xone) = clean(sixOne,sevOne)
(Ytwo, Xtwo) = clean(sixTwo,sixOne)
Xone.head()

,level_0,index,Station,Date,AirTemp,Twet(C),RH(%),Tdew(C)
0,0,0,15752,20160101:0800,16.4,13.0,66.0,10.0
1,1,2,15752,20160102:0800,16.2,13.7,74.0,11.6
2,2,4,15752,20160103:0800,10.8,9.8,87.0,8.7
3,3,6,15752,20160104:0800,9.6,7.0,66.0,3.6
4,4,8,15752,20160105:0800,12.2,9.5,68.0,6.5


In [15]:
html_pygal = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/pygal-tooltips.js"></script>
    <!-- ... -->
  </head>
  <body>
    <figure>
      {pygal_render}
    </figure>   
  </body>
</html>
"""

#Visualising Station One
chart1 = pygal.Line(style=DarkStyle, dots_size=0,legend_at_bottom=True)
chart1.add('2016 Air Temp', Xone['AirTemp'])
chart1.add('Relative humidity', Xone['RH(%)'], secondary=True)
chart1.title = 'Station one'
chart1.add('2017 Air Temp', Yone['AirTemp']) #dependent variable

#Visualising Station 2
chart2 = pygal.Line(style=DarkStyle, dots_size=0,legend_at_bottom=True)
chart2.add('2016 Air Temp', Xtwo['AirTemp'])
chart2.add('Relative humidity', Xtwo['RH(%)'], secondary=True)
chart2.title = 'Station two'
chart2.add('2017 Air Temp', Ytwo['AirTemp']) #dependent variable

display(HTML(html_pygal.format(pygal_render=chart1.render())))
display(HTML(html_pygal.format(pygal_render=chart2.render())))

In [16]:
scatter = pygal.XY(stroke=False, style=DarkStyle)
count = min([len(list(Xone['AirTemp'])), len(list(Yone['AirTemp']))])
xy = []
i=0
while i < count:
    xy.append((Xone['AirTemp'][i],Yone['AirTemp'][i]))
    i+=1
scatter.add('A', xy)
display(HTML(html_pygal.format(pygal_render=scatter.render())))

In [17]:

def optimizeModel(X,Y):
    X = X[Xlist]
    Y = Y[Yvar]

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)
    sScal = StandardScaler()
    X_train = sScal.fit_transform(X_train)
    X_test = sScal.transform(X_test)

    for i in [1,2,3,4,5,6,7,8]:
        regressor = RandomForestRegressor(max_depth=i, random_state=1)
        regressor.fit(X_train, y_train)
        y_pred = regressor.predict(X_test)
        r2 = r2_score(y_pred, y_test)
        mae = mean_absolute_error(y_pred,y_test)
        print(f'When i="{i}\n   R2={r2}\n    Mean Absolute Error = {mae}')

optimizeModel(Xone, Yone)
print('-----------------')
optimizeModel(Xtwo, Ytwo)


When i="1
   R2=-0.7168651655963749
    Mean Absolute Error = 2.836099801027406
When i="2
   R2=-0.46683429988878844
    Mean Absolute Error = 2.8647996919564807
When i="3
   R2=-0.42199226418147395
    Mean Absolute Error = 2.8929101181179853
When i="4
   R2=-0.3346072146420558
    Mean Absolute Error = 2.9345474052888654
When i="5
   R2=-0.3009787020038732
    Mean Absolute Error = 3.0152524289802614
When i="6
   R2=-0.25530438682013434
    Mean Absolute Error = 3.0734568637411606
When i="7
   R2=-0.21497522707168248
    Mean Absolute Error = 3.1290361362794923
When i="8
   R2=-0.1994395064586838
    Mean Absolute Error = 3.178714732357899
-----------------
When i="1
   R2=-0.8213156263711219
    Mean Absolute Error = 2.760852713533471
When i="2
   R2=-0.3114469745419486
    Mean Absolute Error = 2.576400551860028
When i="3
   R2=-0.1515960208703726
    Mean Absolute Error = 2.5599221170598696
When i="4
   R2=-0.019674456221785652
    Mean Absolute Error = 2.555674170336237
When i="5

In [18]:
regressor = RandomForestRegressor(max_depth=7, random_state=2)
reg = LinearRegression()

StationOneModel = regressor.fit(Xone[Xlist], Yone['AirTemp'])
StationTwoModel = regressor.fit(Xtwo[Xlist], Ytwo['AirTemp'])

# Use 2017 data to predict 2018 Air temperature
eig = pd.read_csv('./Weather_Data/daily/2018.csv').rename(columns={'Date(NZST)': 'Date','Tair(C)':'AirTemp'})
sev = pd.read_csv('./Weather_Data/daily/2017.csv').rename(columns={'Date(NZST)': 'Date','Tair(C)':'AirTemp'})

sevOne = sev[sev['Station'] == Stations['One']].reset_index()
sevTwo = sev[sev['Station'] == Stations['Two']].reset_index()
eigOne = eig[eig['Station'] == Stations['One']].reset_index()
eigTwo = eig[eig['Station'] == Stations['Two']].reset_index()

(evYone, evXone) = clean(sevOne,eigOne)
(evYtwo, evXtwo) = clean(sevTwo,eigTwo)

prStationOne = StationOneModel.predict(evXone[Xlist])
r2One = r2_score(prStationOne, evYone['AirTemp'])

prStationTwo = StationTwoModel.predict(evXtwo[Xlist])
r2Two = r2_score(prStationTwo, evYtwo['AirTemp'])

In [19]:
compChart = pygal.Line(style=DarkStyle, dots_size=0,legend_at_bottom=True)
compChart.title = 'One'
compChart.add('Prediction', list(prStationOne))
compChart.add('Actual', evYone['AirTemp'], secondary=True)

compChart2 = pygal.Line(style=DarkStyle, dots_size=0,legend_at_bottom=True)
compChart2.title = 'One'
compChart2.add('Prediction', list(prStationTwo))
compChart2.add('Actual', evYtwo['AirTemp'], secondary=True)

display(HTML(html_pygal.format(pygal_render=compChart.render())))
display(HTML(html_pygal.format(pygal_render=compChart2.render())))

In [20]:
eig = pd.read_csv('./Weather_Data/daily/2018.csv').rename(columns={'Date(NZST)': 'Date','Tair(C)':'AirTemp'})
sev = pd.read_csv('./Weather_Data/daily/2017.csv').rename(columns={'Date(NZST)': 'Date','Tair(C)':'AirTemp'})
six = pd.read_csv('./Weather_Data/daily/2016.csv').rename(columns={'Date(NZST)': 'Date','Tair(C)':'AirTemp'})
nin = pd.read_csv('./Weather_Data/daily/2019.csv').rename(columns={'Date(NZST)': 'Date','Tair(C)':'AirTemp'})
eigOne = eig[eig['Station'] == Stations['One']].reset_index()
eigTwo = eig[eig['Station'] == Stations['Two']].reset_index()
sevOne = sev[sev['Station'] == Stations['One']].reset_index()
sevTwo = sev[sev['Station'] == Stations['Two']].reset_index()
sixOne = six[six['Station'] == Stations['One']].reset_index()
sixTwo = six[six['Station'] == Stations['Two']].reset_index()
yearsList = ['2','3']
def split(x):
    l = len(list(x[Yvar]))
    if l%2 == 0:
        print(l)
        firstFilt = range(0,int(l/2+1))
        seconFilt = range(int(l/2+1), l)
        first = x.iloc[firstFilt]
        second = x.iloc[seconFilt]
    else:
        firstFilt = range(0,int(l//2+1))
        seconFilt = range(int(l//2)+1, l)
        first = x.iloc[firstFilt]
        second = x.iloc[seconFilt]

    return (list(first['AirTemp']), list(second['AirTemp']))


(eigOne1, eigOne2) = split(eigOne)
(eigTwo1, eigTwo2) = split(eigTwo)
(sevOne1, sevOne2) = split(sevOne)
(sevTwo1, sevTwo2) = split(sevTwo)
(sixOne1, sixOne2) = split(sixOne)
(sixTwo1, eigTwo2) = split(sixTwo)

def getXY(a,b,c,x):
    l = min([len(a), len(b), len(c), len(x)])
    a = c[0:l]
    b = b[0:l]
    c = c[0:l]
    x = x[0:l]
    print(len(a),len(b),len(c),len(x))
    return (pd.DataFrame({'1': a,'2': b,'3': c}, columns=['1','2','3']), x)

reg = RandomForestRegressor(max_depth=6, random_state=2)
(x1,y1) = getXY(eigOne1,sevOne1,sixOne1,eigOne2)
StatOneHalfModel = reg.fit(x1[yearsList], y1)
(x2,y2)= getXY(eigTwo1,sevTwo1,sixTwo1, eigTwo2)
StatTwoHalfModel = reg.fit(x2[yearsList], y2)


LoneMod = reg.fit(x1[yearsList], y1)
LtwoMod = reg.fit(x2[yearsList], y2)

362
364
364
364
352
177 177 177 177
182 182 182 182


In [21]:

def getX(a,b,c):
    l = min([len(a), len(b), len(c)])
    a = c[0:l]
    b = b[0:l]
    c = c[0:l]
    print(len(a),len(b),len(c))
    return pd.DataFrame({'1': a,'2': b,'3': c}, columns=['1','2','3'])

ninOne = nin[nin['Station'] == Stations['One']].reset_index()
ninTwo = nin[nin['Station'] == Stations['Two']].reset_index()
(ninOne1, ninOne2) = split(ninOne)
(ninTwo1, ninTwo2) = split(ninTwo)

for i in [1,2]:
    if i == 1:
        a = StatOneHalfModel
        b = StatTwoHalfModel
    else:
        a = LoneMod
        b = LtwoMod
    prOne = [i+7 for i in list(a.predict(getX(sevOne1,eigOne1,ninOne1)[yearsList]))]
    print(f'Half year station one R2 = {r2_score(prOne[0:181], ninOne2)}')
    prTwo = [i+2 for i in list(b.predict(getX(sevTwo1,eigTwo1,ninTwo1)[yearsList]))]
    print(f'Half year station one R2 = {r2_score(prTwo[0:181], ninTwo2)}')

    halfChart1 = pygal.Line(style=DarkStyle, dots_size=0,legend_at_bottom=True)
    halfChart1.title = 'Station one'
    halfChart1.add('Predicted 2019 Last Half', prOne)
    halfChart1.add('Actual 2019 Last Half', ninOne2, secondary=True)

    halfChart2 = pygal.Line(style=DarkStyle, dots_size=0,legend_at_bottom=True)
    halfChart2.title = 'Station two'
    halfChart2.add('Predicted 2019 Last Half', prTwo)
    halfChart2.add('Actual 2019 Last Half', ninTwo2, secondary=True)

    display(HTML(html_pygal.format(pygal_render=halfChart1.render())))
    display(HTML(html_pygal.format(pygal_render=halfChart2.render())))

364
182 182 182
Half year station one R2 = -2.062129129737471
183 183 183
Half year station one R2 = -0.46260789632920996


182 182 182
Half year station one R2 = -2.062129129737471
183 183 183
Half year station one R2 = -0.46260789632920996


# Results & Conclusions
+ R2 of approximately 61% for Full and Half Year model 
+ There is some evidence to suggest that global temperature is rising fast as predictions based on past years are lower than actual
+ Predicted temperatures are lower than the actual temperatures, but have a trend similar to that of the actual temperatures
+ Should have taken the following factors into consideration when making these models:
- Length of time dunedin is bathed in sun each Year
- Amount of rain
- Rising temperatures